<a href="https://colab.research.google.com/github/OseiasBeu/Data-Science/blob/master/Categorical_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
!pip install category-encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 8.4 MB/s 


In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/OseiasBeu/Data-Science/master/DataSets/Chemical_Dependence_Treatment_Program_Admissions__Beginning_2007.csv")

In [3]:
data.head()

,Year,County of Program Location,Program Category,Service Type,Age Group,Primary Substance Group,Admissions
0,2007,Albany,Crisis,Medical Managed Detoxification,Under 18,Heroin,4
1,2007,Albany,Crisis,Medical Managed Detoxification,55 and Older,Alcohol,192
2,2007,Albany,Crisis,Medical Managed Detoxification,55 and Older,All Others,1
3,2007,Albany,Crisis,Medical Managed Detoxification,55 and Older,Heroin,30
4,2007,Albany,Crisis,Medical Managed Detoxification,55 and Older,Other Opioids,4


In [5]:
data.columns

Index(['Year', 'County of Program Location', 'Program Category',
       'Service Type', 'Age Group', 'Primary Substance Group', 'Admissions'],
      dtype='object')

In [ ]:
data['Year'].value_counts().sort_index()

2007    6549
2008    6519
2009    6540
2010    6655
2011    6623
2012    6700
2013    6619
2014    6507
2015    6506
2016    6428
2017    6817
Name: Year, dtype: int64

In [9]:
df_train = data[data['Year'] <= 2013]
df_val = data[data['Year'] > 2013]

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# One Hot Encoder

In [11]:
df_train.shape

(46212, 7)

In [10]:
from category_encoders.one_hot import OneHotEncoder
enc = OneHotEncoder(cols=['County of Program Location', 'Program Category', 'Service Type', 'Age Group', 'Primary Substance Group'], use_cat_names=True)
enc.fit(df_train)


OneHotEncoder(cols=['County of Program Location', 'Program Category',
                    'Service Type', 'Age Group', 'Primary Substance Group'],
              use_cat_names=True)

In [ ]:
df_train.head()

,Year,County of Program Location,Program Category,Service Type,Age Group,Primary Substance Group,Admissions
0,2009,Suffolk,Residential,Intensive Residential,Under 18,Cocaine incl Crack,1
1,2008,Onondaga,Crisis,Med Sup Withdrawal - Inpatient,Under 18,Alcohol,5
2,2009,New York,Outpatient,Outpat Chem Depend for Youth,Under 18,Marijuana incl Hashish,18
3,2007,Orange,Crisis,Medically Monitored Withdrawal,Under 18,Cocaine incl Crack,2
4,2007,Suffolk,Outpatient,Outpatient Rehabilitation,Under 18,Alcohol,2


In [ ]:
df_train['County of Program Location'].nunique()

61

In [ ]:
df_train_ohe = enc.transform(df_train)

In [ ]:
df_train.head()

,Year,County of Program Location,Program Category,Service Type,Age Group,Primary Substance Group,Admissions
0,2009,Suffolk,Residential,Intensive Residential,Under 18,Cocaine incl Crack,1
1,2008,Onondaga,Crisis,Med Sup Withdrawal - Inpatient,Under 18,Alcohol,5
2,2009,New York,Outpatient,Outpat Chem Depend for Youth,Under 18,Marijuana incl Hashish,18
3,2007,Orange,Crisis,Medically Monitored Withdrawal,Under 18,Cocaine incl Crack,2
4,2007,Suffolk,Outpatient,Outpatient Rehabilitation,Under 18,Alcohol,2


In [ ]:
df_train_ohe.head()

,Year,County of Program Location_Suffolk,County of Program Location_Onondaga,County of Program Location_New York,County of Program Location_Orange,County of Program Location_Dutchess,County of Program Location_Queens,County of Program Location_Bronx,County of Program Location_Greene,County of Program Location_Ontario,County of Program Location_Ulster,County of Program Location_Jefferson,County of Program Location_Sullivan,County of Program Location_Kings,County of Program Location_Oneida,County of Program Location_Niagara,County of Program Location_Broome,County of Program Location_Columbia,County of Program Location_Monroe,County of Program Location_Nassau,County of Program Location_Albany,County of Program Location_Westchester,County of Program Location_Putnam,County of Program Location_Fulton,County of Program Location_Richmond,County of Program Location_Rockland,County of Program Location_Erie,County of Program Location_Steuben,County of Program Location_Schenectady,County of Program Location_Delaware,County of Program Location_Cattaraugus,County of Program Location_Montgomery,County of Program Location_Warren,County of Program Location_Seneca,County of Program Location_Genesee,County of Program Location_Chemung,County of Program Location_Madison,County of Program Location_Chautauqua,County of Program Location_Orleans,County of Program Location_Herkimer,County of Program Location_Rensselaer,County of Program Location_Tompkins,County of Program Location_Tioga,County of Program Location_Oswego,County of Program Location_St Lawrence,County of Program Location_Chenango,County of Program Location_Livingston,County of Program Location_Yates,County of Program Location_Cortland,County of Program Location_Allegany,County of Program Location_Cayuga,County of Program Location_Saratoga,County of Program Location_Washington,County of Program Location_Schoharie,County of Program Location_Wayne,County of Program Location_Wyoming,County of Program Location_Franklin,County of Program Location_Essex,County of Program Location_Lewis,County of Program Location_Otsego,County of Program Location_Clinton,County of Program Location_Schuyler,Program Category_Residential,Program Category_Crisis,Program Category_Outpatient,Program Category_Inpatient,Program Category_Opioid Treatment Program,Service Type_Intensive Residential,Service Type_Med Sup Withdrawal - Inpatient,Service Type_Outpat Chem Depend for Youth,Service Type_Medically Monitored Withdrawal,Service Type_Outpatient Rehabilitation,Service Type_Community Residential,Service Type_Outpatient Clinic,Service Type_Inpatient Rehabilitation,Service Type_Methadone Clinic,Service Type_Supportive Living,Service Type_OTP Intensive Residential,Service Type_Medically Managed Detoxification,Service Type_Res Rehab for Youth,Service Type_Meth to Abst - Residential,Service Type_Specialized OP - TBI,Service Type_Med Sup Withdrawal - Outpatient,Service Type_Specialized Services OP Rehab,Service Type_Specialized OP - Mobile,Service Type_Long Term Res CD/Youth,Service Type_Limited Outpatient/KEEP,Service Type_Short Term Res CD/Youth,Service Type_Non-Med Sup Chem Dep OP,Age Group_Under 18,Age Group_55 and Older,Age Group_18 thru 24,Age Group_25 thru 34,Age Group_45 thru 54,Age Group_35 thru 44,Primary Substance Group_Cocaine incl Crack,Primary Substance Group_Alcohol,Primary Substance Group_Marijuana incl Hashish,Primary Substance Group_Heroin,Primary Substance Group_All Others,Primary Substance Group_Other Opioids,Admissions
0,2009,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
1,2008,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,5
2,2009,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
df_val_ohe = enc.transform(df_val)

In [ ]:
df_val_ohe.head()

,Year,County of Program Location_Suffolk,County of Program Location_Onondaga,County of Program Location_New York,County of Program Location_Orange,County of Program Location_Dutchess,County of Program Location_Queens,County of Program Location_Bronx,County of Program Location_Greene,County of Program Location_Ontario,County of Program Location_Ulster,County of Program Location_Jefferson,County of Program Location_Sullivan,County of Program Location_Kings,County of Program Location_Oneida,County of Program Location_Niagara,County of Program Location_Broome,County of Program Location_Columbia,County of Program Location_Monroe,County of Program Location_Nassau,County of Program Location_Albany,County of Program Location_Westchester,County of Program Location_Putnam,County of Program Location_Fulton,County of Program Location_Richmond,County of Program Location_Rockland,County of Program Location_Erie,County of Program Location_Steuben,County of Program Location_Schenectady,County of Program Location_Delaware,County of Program Location_Cattaraugus,County of Program Location_Montgomery,County of Program Location_Warren,County of Program Location_Seneca,County of Program Location_Genesee,County of Program Location_Chemung,County of Program Location_Madison,County of Program Location_Chautauqua,County of Program Location_Orleans,County of Program Location_Herkimer,County of Program Location_Rensselaer,County of Program Location_Tompkins,County of Program Location_Tioga,County of Program Location_Oswego,County of Program Location_St Lawrence,County of Program Location_Chenango,County of Program Location_Livingston,County of Program Location_Yates,County of Program Location_Cortland,County of Program Location_Allegany,County of Program Location_Cayuga,County of Program Location_Saratoga,County of Program Location_Washington,County of Program Location_Schoharie,County of Program Location_Wayne,County of Program Location_Wyoming,County of Program Location_Franklin,County of Program Location_Essex,County of Program Location_Lewis,County of Program Location_Otsego,County of Program Location_Clinton,County of Program Location_Schuyler,Program Category_Residential,Program Category_Crisis,Program Category_Outpatient,Program Category_Inpatient,Program Category_Opioid Treatment Program,Service Type_Intensive Residential,Service Type_Med Sup Withdrawal - Inpatient,Service Type_Outpat Chem Depend for Youth,Service Type_Medically Monitored Withdrawal,Service Type_Outpatient Rehabilitation,Service Type_Community Residential,Service Type_Outpatient Clinic,Service Type_Inpatient Rehabilitation,Service Type_Methadone Clinic,Service Type_Supportive Living,Service Type_OTP Intensive Residential,Service Type_Medically Managed Detoxification,Service Type_Res Rehab for Youth,Service Type_Meth to Abst - Residential,Service Type_Specialized OP - TBI,Service Type_Med Sup Withdrawal - Outpatient,Service Type_Specialized Services OP Rehab,Service Type_Specialized OP - Mobile,Service Type_Long Term Res CD/Youth,Service Type_Limited Outpatient/KEEP,Service Type_Short Term Res CD/Youth,Service Type_Non-Med Sup Chem Dep OP,Age Group_Under 18,Age Group_55 and Older,Age Group_18 thru 24,Age Group_25 thru 34,Age Group_45 thru 54,Age Group_35 thru 44,Primary Substance Group_Cocaine incl Crack,Primary Substance Group_Alcohol,Primary Substance Group_Marijuana incl Hashish,Primary Substance Group_Heroin,Primary Substance Group_All Others,Primary Substance Group_Other Opioids,Admissions
12,2015,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
18,2014,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1
23,2014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0

In [ ]:
X_train = df_train_ohe.drop("Admissions", axis=1)
X_val = df_val_ohe.drop("Admissions", axis=1)

y_train = df_train_ohe["Admissions"]
y_val = df_val_ohe["Admissions"]

In [ ]:
X_train.head()

,Year,County of Program Location_Suffolk,County of Program Location_Onondaga,County of Program Location_New York,County of Program Location_Orange,County of Program Location_Dutchess,County of Program Location_Queens,County of Program Location_Bronx,County of Program Location_Greene,County of Program Location_Ontario,County of Program Location_Ulster,County of Program Location_Jefferson,County of Program Location_Sullivan,County of Program Location_Kings,County of Program Location_Oneida,County of Program Location_Niagara,County of Program Location_Broome,County of Program Location_Columbia,County of Program Location_Monroe,County of Program Location_Nassau,County of Program Location_Albany,County of Program Location_Westchester,County of Program Location_Putnam,County of Program Location_Fulton,County of Program Location_Richmond,County of Program Location_Rockland,County of Program Location_Erie,County of Program Location_Steuben,County of Program Location_Schenectady,County of Program Location_Delaware,County of Program Location_Cattaraugus,County of Program Location_Montgomery,County of Program Location_Warren,County of Program Location_Seneca,County of Program Location_Genesee,County of Program Location_Chemung,County of Program Location_Madison,County of Program Location_Chautauqua,County of Program Location_Orleans,County of Program Location_Herkimer,County of Program Location_Rensselaer,County of Program Location_Tompkins,County of Program Location_Tioga,County of Program Location_Oswego,County of Program Location_St Lawrence,County of Program Location_Chenango,County of Program Location_Livingston,County of Program Location_Yates,County of Program Location_Cortland,County of Program Location_Allegany,County of Program Location_Cayuga,County of Program Location_Saratoga,County of Program Location_Washington,County of Program Location_Schoharie,County of Program Location_Wayne,County of Program Location_Wyoming,County of Program Location_Franklin,County of Program Location_Essex,County of Program Location_Lewis,County of Program Location_Otsego,County of Program Location_Clinton,County of Program Location_Schuyler,Program Category_Residential,Program Category_Crisis,Program Category_Outpatient,Program Category_Inpatient,Program Category_Opioid Treatment Program,Service Type_Intensive Residential,Service Type_Med Sup Withdrawal - Inpatient,Service Type_Outpat Chem Depend for Youth,Service Type_Medically Monitored Withdrawal,Service Type_Outpatient Rehabilitation,Service Type_Community Residential,Service Type_Outpatient Clinic,Service Type_Inpatient Rehabilitation,Service Type_Methadone Clinic,Service Type_Supportive Living,Service Type_OTP Intensive Residential,Service Type_Medically Managed Detoxification,Service Type_Res Rehab for Youth,Service Type_Meth to Abst - Residential,Service Type_Specialized OP - TBI,Service Type_Med Sup Withdrawal - Outpatient,Service Type_Specialized Services OP Rehab,Service Type_Specialized OP - Mobile,Service Type_Long Term Res CD/Youth,Service Type_Limited Outpatient/KEEP,Service Type_Short Term Res CD/Youth,Service Type_Non-Med Sup Chem Dep OP,Age Group_Under 18,Age Group_55 and Older,Age Group_18 thru 24,Age Group_25 thru 34,Age Group_45 thru 54,Age Group_35 thru 44,Primary Substance Group_Cocaine incl Crack,Primary Substance Group_Alcohol,Primary Substance Group_Marijuana incl Hashish,Primary Substance Group_Heroin,Primary Substance Group_All Others,Primary Substance Group_Other Opioids
0,2009,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,2008,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,2009,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
mdl = RandomForestRegressor(n_jobs=6, n_estimators=100, random_state=22)
mdl.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=6,
           oob_score=False, random_state=22, verbose=0, warm_start=False)

In [ ]:
p_ohe = mdl.predict(X_val)

In [ ]:
p_ohe[:5]

array([ 3.78,  3.06, 18.99,  3.19, 36.27])

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, p_ohe)

14.194968009749408

# Ordinal

In [ ]:
df_train.head()

,Year,County of Program Location,Program Category,Service Type,Age Group,Primary Substance Group,Admissions
0,2009,Suffolk,Residential,Intensive Residential,Under 18,Cocaine incl Crack,1
1,2008,Onondaga,Crisis,Med Sup Withdrawal - Inpatient,Under 18,Alcohol,5
2,2009,New York,Outpatient,Outpat Chem Depend for Youth,Under 18,Marijuana incl Hashish,18
3,2007,Orange,Crisis,Medically Monitored Withdrawal,Under 18,Cocaine incl Crack,2
4,2007,Suffolk,Outpatient,Outpatient Rehabilitation,Under 18,Alcohol,2


In [ ]:
from category_encoders.ordinal import OrdinalEncoder
enc = OrdinalEncoder(cols=['County of Program Location', 'Program Category', 'Service Type', 'Age Group', 'Primary Substance Group'])
enc.fit(df_train)
df_train_ord = enc.transform(df_train)

In [ ]:
df_train_ord.head()

,Year,County of Program Location,Program Category,Service Type,Age Group,Primary Substance Group,Admissions
0,2009,1,1,1,1,1,1
1,2008,2,2,2,1,2,5
2,2009,3,3,3,1,3,18
3,2007,4,2,4,1,1,2
4,2007,1,3,5,1,2,2


In [ ]:
df_val_ord = enc.transform(df_val)

In [ ]:
df_val_ord.head()

,Year,County of Program Location,Program Category,Service Type,Age Group,Primary Substance Group,Admissions
12,2015,1,3,5.0,1,4,1
18,2014,4,2,4.0,1,3,1
23,2014,26,2,2.0,1,2,1
39,2015,9,2,4.0,1,4,1
50,2014,2,2,2.0,1,4,2


In [ ]:
X_train = df_train_ord.drop("Admissions", axis=1)
X_val = df_val_ord.drop("Admissions", axis=1)

y_train = df_train_ord["Admissions"]
y_val = df_val_ord["Admissions"]

mdl = RandomForestRegressor(n_jobs=6, n_estimators=100, random_state=22)
mdl.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=6,
           oob_score=False, random_state=22, verbose=0, warm_start=False)

In [ ]:
p_ord = mdl.predict(X_val)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, p_ord)

14.790480996267807

# Target Encoding

In [ ]:
df_train.head()

,Year,County of Program Location,Program Category,Service Type,Age Group,Primary Substance Group,Admissions
0,2009,Suffolk,Residential,Intensive Residential,Under 18,Cocaine incl Crack,1
1,2008,Onondaga,Crisis,Med Sup Withdrawal - Inpatient,Under 18,Alcohol,5
2,2009,New York,Outpatient,Outpat Chem Depend for Youth,Under 18,Marijuana incl Hashish,18
3,2007,Orange,Crisis,Medically Monitored Withdrawal,Under 18,Cocaine incl Crack,2
4,2007,Suffolk,Outpatient,Outpatient Rehabilitation,Under 18,Alcohol,2


In [ ]:
from category_encoders.target_encoder import TargetEncoder
enc = TargetEncoder(cols=['County of Program Location', 'Program Category', 'Service Type', 'Age Group', 'Primary Substance Group'],
                   min_samples_leaf=200)
enc.fit(df_train, df_train['Admissions'])
df_train_trg = enc.transform(df_train)

In [ ]:
df_train_trg.head()

,Year,County of Program Location,Program Category,Service Type,Age Group,Primary Substance Group,Admissions
0,2009,84.365213,12.540684,26.125614,25.444371,33.193930,1
1,2008,40.103075,68.736003,63.489893,25.444371,96.876860,5
2,2009,145.838906,57.269020,45.819998,25.444371,48.186532,18
3,2007,29.188417,68.736003,42.527388,25.444371,33.193930,2
4,2007,84.365213,57.269020,14.000000,25.444371,96.876860,2


In [ ]:
df_val_trg = enc.transform(df_val)

In [ ]:
X_train = df_train_trg.drop("Admissions", axis=1)
X_val = df_val_trg.drop("Admissions", axis=1)

y_train = df_train_trg["Admissions"]
y_val = df_val_trg["Admissions"]

mdl = RandomForestRegressor(n_jobs=6, n_estimators=100, random_state=22)
mdl.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=6,
           oob_score=False, random_state=22, verbose=0, warm_start=False)

In [ ]:
p_trg = mdl.predict(X_val)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, p_trg)

14.180224851897199

In [ ]:
?TargetEncoder

Init signature:
TargetEncoder(
    ['verbose=0', 'cols=None', 'drop_invariant=False', 'return_df=True', "handle_missing='value'", "handle_unknown='value'", 'min_samples_leaf=1', 'smoothing=1.0'],
)
Docstring:     
Target encoding for categorical features.

For the case of categorical target: features are replaced with a blend of posterior probability of the target
given particular categorical value and the prior probability of the target over all the training data.

For the case of continuous target: features are replaced with a blend of the expected value of the target
given particular categorical value and the expected value of the target over all the training data.

Parameters
----------

verbose: int
    integer indicating verbosity of the output. 0 for none.
cols: list
    a list of columns to encode, if None, all string columns will be encoded.
drop_invariant: bool
    boolean for whether or not to drop columns with 0 variance.
return_df: bool
    boolean for whether to return a 